# π-Proteoformer PTM Sites Prediction 

This notebook demonstrates how to use the pre-trained $\pi$-Proteoformer model for predicting phosphorylation sites (S/T).

## 1. Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path

# Add the proteoformer package to path
PROJECT_ROOT = Path("your/project/root")
sys.path.insert(0, str(PROJECT_ROOT  / "pi-Proteoformer" / "src"))

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm

# Import Proteoformer components
from proteoformer.net import ProteoformerForEmbedding
from proteoformer.tokenization import ProteoformerTokenizer
from proteoformer.models import ProteoformerClassifier

print("All imports successful!")

All imports successful!


: 

## 2. Configuration

In [2]:
# Paths configuration
PRETRAINED_MODEL_PATH = '/pretrain_stage2/checkpoint-197000'

# Path to the fine-tuned phosphorylation S/T checkpoint
PTM_CHECKPOINT_PATH = 'phosphorylation_st_best_checkpoint_proteoformer.pt'

# Example test data path (optional)
TEST_DATA_PATH = '/data/phosphorylation_st_test_data_filtered.tsv'

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Model parameters
MAX_LENGTH = 31  # Default sequence length (15 residues on each side + center)
HIDDEN_SIZE = 512  # CNN classifier hidden size

Using device: cuda


## 3. Load Models

In [3]:
def load_ptm_predictor(pretrained_path, checkpoint_path, device, hidden_size=512):
    """
    Load the complete PTM prediction model with pre-trained weights.
    
    Args:
        pretrained_path: Path to the pre-trained Proteoformer2 model
        checkpoint_path: Path to the fine-tuned PTM classifier checkpoint
        device: Torch device to load the model on
        hidden_size: Hidden size for the CNN classifier
        
    Returns:
        model: Loaded ProteoformerClassifier model
        tokenizer: Loaded ProteoformerTokenizer
    """
    # print(f"Loading tokenizer from {pretrained_path}...")
    tokenizer = ProteoformerTokenizer.from_pretrained(str(pretrained_path))
    
    # print(f"Loading Proteoformer2 backbone model from {pretrained_path}...")
    proteoformer_model = ProteoformerForEmbedding.from_pretrained(str(pretrained_path))
    
    # Get embedding dimension from model config
    embedding_dim = proteoformer_model.config.hidden_size
    print(f"Model hidden size: {embedding_dim}")
    print(f"Number of layers: {proteoformer_model.config.num_hidden_layers}")
    
    # Create the classifier model
    print("Creating ProteoformerClassifier...")
    model = ProteoformerClassifier(proteoformer_model, embedding_dim, hidden_size)
    
    # Load the fine-tuned checkpoint
    # print(f"Loading fine-tuned checkpoint from {checkpoint_path}...")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Load state dict
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Print checkpoint information
    if 'metrics' in checkpoint:
        metrics = checkpoint['metrics']
        print(f"Checkpoint metrics:")
        print(f"  - MCC: {metrics.get('MCC', 'N/A')}")
        print(f"  - AUC-ROC: {metrics.get('AUC_ROC', 'N/A')}")
        print(f"  - F1 Score: {metrics.get('F1_Score', 'N/A')}")
    
    # Move to device and set to eval mode
    model = model.to(device)
    model.eval()
    
    print("Model loaded successfully!")
    return model, tokenizer

In [4]:
# Load the model
model, tokenizer = load_ptm_predictor(
    pretrained_path=PRETRAINED_MODEL_PATH,
    checkpoint_path=PTM_CHECKPOINT_PATH,
    device=DEVICE,
    hidden_size=HIDDEN_SIZE
)

You are using a model of type proteoformer2 to instantiate a model of type proteoformer. This is not supported for all configurations of models and can yield errors.


Model hidden size: 1280
Number of layers: 24
Creating ProteoformerClassifier...
Checkpoint metrics:
  - MCC: 0.5546809420962806
  - AUC-ROC: 0.882858683222495
  - F1 Score: 0.6237148732008225
Model loaded successfully!


## 4. Define Prediction Functions

In [5]:
def predict_single_sequence(model, tokenizer, sequence, device, max_length=31, threshold=0.5):
    """
    Predict phosphorylation probability for a single sequence window.
    
    Args:
        model: Loaded ProteoformerClassifier model
        tokenizer: Loaded ProteoformerTokenizer
        sequence: Protein sequence window (e.g., 31 amino acids with S/T at center)
        device: Torch device
        max_length: Maximum sequence length for tokenization
        threshold: Classification threshold (default 0.5)
        
    Returns:
        probability: Phosphorylation probability (0-1)
        prediction: Binary prediction (0 or 1)
    """
    model.eval()
    
    with torch.no_grad():
        # Tokenize the sequence
        enc = tokenizer(
            sequence, 
            return_tensors="pt", 
            padding=True, 
            truncation=True,
            max_length=max_length,
            add_special_tokens=False
        )
        
        input_ids = enc["input_ids"].to(device)
        attention_mask = enc.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(device)
        
        # Get prediction probability
        probability = model(input_ids, attention_mask, return_logits=False)
        probability = probability.item()
        
        # Get binary prediction
        prediction = 1 if probability > threshold else 0
        
    return probability, prediction


def predict_batch(model, tokenizer, sequences, device, max_length=31, batch_size=32, threshold=0.5):
    """
    Predict phosphorylation probabilities for a batch of sequences.
    
    Args:
        model: Loaded ProteoformerClassifier model
        tokenizer: Loaded ProteoformerTokenizer
        sequences: List of protein sequence windows
        device: Torch device
        max_length: Maximum sequence length for tokenization
        batch_size: Batch size for inference
        threshold: Classification threshold (default 0.5)
        
    Returns:
        probabilities: List of phosphorylation probabilities
        predictions: List of binary predictions
    """
    model.eval()
    all_probs = []
    all_preds = []
    
    # Process in batches
    for i in tqdm(range(0, len(sequences), batch_size), desc="Predicting"):
        batch_seqs = sequences[i:i+batch_size]
        
        with torch.no_grad():
            # Tokenize batch
            enc = tokenizer(
                batch_seqs,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=max_length,
                add_special_tokens=False
            )
            
            input_ids = enc["input_ids"].to(device)
            attention_mask = enc.get("attention_mask", None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)
            
            # Get prediction probabilities
            probs = model(input_ids, attention_mask, return_logits=False)
            probs = probs.cpu().numpy()
            
            all_probs.extend(probs.tolist())
            all_preds.extend((probs > threshold).astype(int).tolist())
    
    return all_probs, all_preds


def extract_site_windows(protein_sequence, window_size=16, target_residues=['S', 'T']):
    """
    Extract sequence windows for all potential phosphorylation sites in a protein.
    
    Args:
        protein_sequence: Full protein sequence
        window_size: Number of residues on each side of the target site (default 16)
        target_residues: List of target residues to extract windows for (default ['S', 'T'])
        
    Returns:
        List of tuples: (position, residue, sequence_window)
    """
    windows = []
    seq_len = len(protein_sequence)
    
    for i, residue in enumerate(protein_sequence):
        if residue in target_residues:
            # Extract window
            start = max(0, i - window_size)
            end = min(seq_len, i + window_size + 1)
            
            # Pad if necessary
            left_pad = window_size - (i - start)
            right_pad = window_size - (end - i - 1)
            
            window = 'X' * left_pad + protein_sequence[start:end] + 'X' * right_pad
            
            windows.append((i + 1, residue, window))  # Position is 1-based
    
    return windows

## 5. Example: Single Sequence Prediction

In [6]:
# Example sequence windows (33 amino acids with S/T at center position)
# These are example phosphorylation site sequences from the test dataset

example_sequences = [
    # Positive examples (known phosphorylation sites)
    "SQVAIEALSAMPTVRSFANEEGEAQKFREKL",  # S at position 16
    "LEFAKENRKCQEQAVSPKVDDQCGNSSSIPF",  # T at position 16
    "GQHETGGSPSKQQMRSVISVTSALKEVGVDS",  # S at position 16
    
    # You can add your own sequences here
]

print("Single sequence prediction examples:")
print("=" * 60)

for seq in example_sequences:
    prob, pred = predict_single_sequence(model, tokenizer, seq, DEVICE)
    center_pos = len(seq) // 2
    center_residue = seq[center_pos]
    
    print(f"\nSequence: {seq}")
    print(f"Center residue: {center_residue} (position {center_pos + 1})")
    print(f"Phosphorylation probability: {prob:.4f}")
    print(f"Prediction: {'Phosphorylated' if pred == 1 else 'Not phosphorylated'}")

Single sequence prediction examples:

Sequence: SQVAIEALSAMPTVRSFANEEGEAQKFREKL
Center residue: S (position 16)
Phosphorylation probability: 0.9241
Prediction: Phosphorylated

Sequence: LEFAKENRKCQEQAVSPKVDDQCGNSSSIPF
Center residue: S (position 16)
Phosphorylation probability: 0.8579
Prediction: Phosphorylated

Sequence: GQHETGGSPSKQQMRSVISVTSALKEVGVDS
Center residue: S (position 16)
Phosphorylation probability: 0.6696
Prediction: Phosphorylated
